<a href="https://colab.research.google.com/github/Sion1225/Study-Deeplearning-NLP/blob/master/Character-Level%20Neura%20Machine%20Translation%20with%20Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset is from "http://www.manythings.org/anki"

# Pre-processing

In [1]:
"""
import os
import shutil
import zipfile
"""

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
"""
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)
"""

"\nhttp = urllib3.PoolManager()\nurl ='http://www.manythings.org/anki/fra-eng.zip'\nfilename = 'fra-eng.zip'\npath = os.getcwd()\nzipfilename = os.path.join(path, filename)\nwith http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       \n    shutil.copyfileobj(r, out_file)\n\nwith zipfile.ZipFile(zipfilename, 'r') as zip_ref:\n    zip_ref.extractall(path)\n"

In [3]:
lines = pd.read_csv("DataSet/spa.txt", names=["src", "tar", "lic"], sep='\t')
del lines["lic"]
print("Count of all semples : ", len(lines))

Count of all semples :  139013


In [4]:
lines = lines[:80000]

In [5]:
print(lines[:5])

   src      tar
0  Go.      Ve.
1  Go.    Vete.
2  Go.    Vaya.
3  Go.  Váyase.
4  Hi.    Hola.


In [6]:
lines.sample(10)

,src,tar
66746,"You don't fool me, you know.",Sabes que no me engañas.
47784,Fill out the application.,Llene usted el pliego de solicitud.
65143,The sun is shining brightly.,El sol está brillando intensamente.
34961,Many people shop here.,Mucha gente compra aquí.
70629,Tom is faster than you think.,Tomás es más rápido de lo que pensás.
23380,You're not my type.,No eres mi tipo.
31888,Were you intimidated?,¿Te sentías intimidado?
54445,I've always hated biology.,Siempre he odiado la biología.
30711,The house is on fire!,¡La casa se está incendiando!
9622,Here's your bag.,Aquí está tu bolsa.


In [7]:
lines.tar = lines.tar.apply(lambda x : "\t" + x + "\n")
lines.sample(10)

,src,tar
32553,You’re so arrogant.,\tEres una presumida.\n
5873,Look at these.,\tFíjate en éstos.\n
12410,Explain it to me.,\tExplicadmelo.\n
76713,He always leaves home at seven.,\tÉl sale siempre a las siete de casa.\n
38717,I like to travel alone.,\tMe gusta viajar solo.\n
74230,She is giving the baby a bath.,\tElla está bañando al bebé.\n
77120,I am anxious about your health.,\tEstoy preocupado por su salud.\n
18424,Tom speaks French.,\tTom habla francés.\n
27758,You know who I mean.,\tSabes a quién me refiero.\n
18714,What did Tom know?,\t¿Qué sabía Tom?\n


In [8]:
src_vocab = set()
for line in lines.src :
    for char in line :
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [9]:
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1
print("Source sentences' char set :",src_vocab_size)
print("Target sentences' char set :",tar_vocab_size)

Source sentences' char set : 81
Target sentences' char set : 100


In [10]:
print(src_vocab)
print(tar_vocab)

{'H', 'E', '.', 'z', 'X', '/', 'h', 'G', 'F', 'l', '°', ':', 't', '9', 'm', 'u', 'y', 'V', 'M', 'T', "'", ',', 'a', 's', '?', '1', 'J', 'v', 'L', 'p', 'O', 'B', 'k', 'n', 'D', 'Y', ' ', 'S', '-', 'q', 'Z', 'Q', 'j', 'g', 'r', 'e', '2', '"', 'P', 'w', 'K', '!', 'U', '6', '3', '’', 'W', '$', 'b', '%', 'C', 'A', 'I', '‘', 'R', '5', 'c', 'o', '\xa0', 'é', '7', '€', 'f', '0', 'x', 'i', 'N', 'd', '8', '4'}
{')', 'H', 'E', '.', '\n', 'z', 'X', '/', 'h', '»', '—', 'G', 'l', 'F', '°', '(', 'º', ':', 't', 'ñ', '9', 'm', 'u', 'V', 'y', 'ó', 'M', 'T', "'", ',', 'a', 's', '+', '?', '1', 'J', 'с', 'v', 'L', 'p', '\t', 'O', 'B', '«', 'É', 'k', 'n', 'D', 'Y', ' ', 'Ú', 'S', '-', 'q', 'í', 'Z', 'Q', 'á', 'j', 'g', 'r', 'e', '¡', '"', '2', 'P', 'Á', 'w', 'K', '!', 'U', '6', '3', 'W', '$', 'b', '%', '¿', 'C', 'A', 'I', 'ü', 'R', '5', 'c', 'o', 'é', 'ú', '7', '€', 'Ó', 'f', '0', 'x', 'i', 'N', 'd', '8', '4'}


In [11]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[26:75])
print(tar_vocab[26:75])

['E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0']
['?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v']


In [12]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, "'": 6, ',': 7, '-': 8, '.': 9, '/': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, '?': 22, 'A': 23, 'B': 24, 'C': 25, 'D': 26, 'E': 27, 'F': 28, 'G': 29, 'H': 30, 'I': 31, 'J': 32, 'K': 33, 'L': 34, 'M': 35, 'N': 36, 'O': 37, 'P': 38, 'Q': 39, 'R': 40, 'S': 41, 'T': 42, 'U': 43, 'V': 44, 'W': 45, 'X': 46, 'Y': 47, 'Z': 48, 'a': 49, 'b': 50, 'c': 51, 'd': 52, 'e': 53, 'f': 54, 'g': 55, 'h': 56, 'i': 57, 'j': 58, 'k': 59, 'l': 60, 'm': 61, 'n': 62, 'o': 63, 'p': 64, 'q': 65, 'r': 66, 's': 67, 't': 68, 'u': 69, 'v': 70, 'w': 71, 'x': 72, 'y': 73, 'z': 74, '\xa0': 75, '°': 76, 'é': 77, '‘': 78, '’': 79, '€': 80}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, "'": 8, '(': 9, ')': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, '?': 27, 'A': 28, 'B': 29, 'C': 30, 'D': 31, 'E': 32, 'F'

In [13]:
encoder_input = []

for line in lines["src"] :
    encoded_line = []

    for char in line :
        encoded_line.append(src_to_index[char])
    
    encoder_input.append(encoded_line)

print("Source sentences : \n", lines["src"][:5])
print("Encoded source sentences : ", encoder_input[:5])

Source sentences : 
 0    Go.
1    Go.
2    Go.
3    Go.
4    Hi.
Name: src, dtype: object
Encoded source sentences :  [[29, 63, 9], [29, 63, 9], [29, 63, 9], [29, 63, 9], [30, 57, 9]]


In [14]:
decoder_input = []

for line in lines["tar"] :
  encoded_line = []

  for char in line :
    encoded_line.append(tar_to_index[char])

  decoder_input.append(encoded_line)

print("Target sentences : \n", lines["tar"][:5])
print("Encoded target sentences : ", decoder_input[:5])

Target sentences : 
 0        \tVe.\n
1      \tVete.\n
2      \tVaya.\n
3    \tVáyase.\n
4      \tHola.\n
Name: tar, dtype: object
Encoded target sentences :  [[1, 49, 58, 14, 2], [1, 49, 58, 73, 58, 14, 2], [1, 49, 54, 78, 54, 14, 2], [1, 49, 90, 78, 54, 72, 58, 14, 2], [1, 35, 68, 65, 54, 14, 2]]


In [15]:
decoder_target = []

for line in lines["tar"] :
    timestep = 0
    encoded_line = []

    for char in line :
        if timestep > 0 :
            encoded_line.append(tar_to_index[char])
        timestep += 1
    
    decoder_target.append(encoded_line)

print("Encoded target label sentences : ", decoder_target[:5])

Encoded target label sentences :  [[49, 58, 14, 2], [49, 58, 73, 58, 14, 2], [49, 54, 78, 54, 14, 2], [49, 90, 78, 54, 72, 58, 14, 2], [35, 68, 65, 54, 14, 2]]


In [16]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('Source sentences maximum length :',max_src_len)
print('Target sentences maximum length :',max_tar_len)

Source sentences maximum length : 31
Target sentences maximum length : 72


In [17]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding="post")
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [18]:
import ray
ray.init()

2023-03-01 15:06:31,089	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.9.13
Ray version:,2.3.0


In [19]:
#One-Hot encoding (Character-level)
@ray.remote
def One_Hot(encoder_input, decoder_input, decoder_target) :
    encoder_input = to_categorical(encoder_input)
    decoder_input = to_categorical(decoder_input)
    decoder_target = to_categorical(decoder_target)

    return encoder_input, decoder_input, decoder_target

encoder_input, decoder_input, decoder_target = ray.get(One_Hot.remote(encoder_input, decoder_input, decoder_target))

In [20]:
print(encoder_input[:3])
ray.shutdown()

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]


In [21]:
print(encoder_input.shape)
print(max_src_len)
print(src_vocab_size)

(80000, 31, 81)
31
81


# Model

### Training

In [22]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [23]:
#Encoder
encoder_inputs = Input(shape=(max_src_len, src_vocab_size))

encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]

In [24]:
#Decoder
decoder_inputs = Input(shape=(None, tar_vocab_size))

decoder_lstm = LSTM(units=256, return_sequences=True , return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(units=tar_vocab_size, activation="softmax")
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [25]:
print(decoder_outputs.shape)

(None, None, 100)


In [26]:
#Model Combine & Compile
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="Nadam", loss="categorical_crossentropy", metrics=["accuracy"])

In [27]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=35, validation_split=0.2) 

Epoch 1/35
1000/1000 [==============================] - 30s 16ms/step - loss: 0.8170 - accuracy: 0.7705 - val_loss: 0.8773 - val_accuracy: 0.7296
Epoch 2/35
1000/1000 [==============================] - 14s 14ms/step - loss: 0.5847 - accuracy: 0.8202 - val_loss: 0.7606 - val_accuracy: 0.7666
Epoch 3/35
1000/1000 [==============================] - 14s 14ms/step - loss: 0.5125 - accuracy: 0.8433 - val_loss: 0.6834 - val_accuracy: 0.7927
Epoch 4/35
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4594 - accuracy: 0.8608 - val_loss: 0.6289 - val_accuracy: 0.8099
Epoch 5/35
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4205 - accuracy: 0.8732 - val_loss: 0.5875 - val_accuracy: 0.8224
Epoch 6/35
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3939 - accuracy: 0.8813 - val_loss: 0.5611 - val_accuracy: 0.8304
Epoch 7/35
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3700 - accuracy: 0.8884 - val_loss: 0.5322 -

In [28]:
#model.save_weight("Models/CharLevel Seq2Seq.ckpt")

### Activating

In [29]:
#define encoder
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [30]:
#define decoder
decoder_state_input_h = Input(shape=(256,)) # hidden
decoder_state_input_c = Input(shape=(256,)) # cell
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

#decoder_lstm = LSTM(units=256, return_sequences=True , return_state=True)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c] # save hidden&cell state

#decoder_softmax_layer = Dense(units=tar_vocab_size, activation="softmax")
decoder_outputs = decoder_softmax_layer(decoder_outputs)

decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [31]:
print(tar_vocab_size)
print(decoder_outputs.shape)

100
(None, None, 100)


In [32]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [33]:
print(index_to_tar)

{1: '\t', 2: '\n', 3: ' ', 4: '!', 5: '"', 6: '$', 7: '%', 8: "'", 9: '(', 10: ')', 11: '+', 12: ',', 13: '-', 14: '.', 15: '/', 16: '0', 17: '1', 18: '2', 19: '3', 20: '4', 21: '5', 22: '6', 23: '7', 24: '8', 25: '9', 26: ':', 27: '?', 28: 'A', 29: 'B', 30: 'C', 31: 'D', 32: 'E', 33: 'F', 34: 'G', 35: 'H', 36: 'I', 37: 'J', 38: 'K', 39: 'L', 40: 'M', 41: 'N', 42: 'O', 43: 'P', 44: 'Q', 45: 'R', 46: 'S', 47: 'T', 48: 'U', 49: 'V', 50: 'W', 51: 'X', 52: 'Y', 53: 'Z', 54: 'a', 55: 'b', 56: 'c', 57: 'd', 58: 'e', 59: 'f', 60: 'g', 61: 'h', 62: 'i', 63: 'j', 64: 'k', 65: 'l', 66: 'm', 67: 'n', 68: 'o', 69: 'p', 70: 'q', 71: 'r', 72: 's', 73: 't', 74: 'u', 75: 'v', 76: 'w', 77: 'x', 78: 'y', 79: 'z', 80: '¡', 81: '«', 82: '°', 83: 'º', 84: '»', 85: '¿', 86: 'Á', 87: 'É', 88: 'Ó', 89: 'Ú', 90: 'á', 91: 'é', 92: 'í', 93: 'ñ', 94: 'ó', 95: 'ú', 96: 'ü', 97: 'с', 98: '—', 99: '€'}


In [34]:
def decode_sequence(input_seq) :
    #input input to encoder
    states_value = encoder_model.predict(input_seq)
    
    #making \t(<SOS>) One-Hot vector
    target_seq = np.zeros((1,1,tar_vocab_size))
    target_seq[0,0, tar_to_index['\t']] = 1
    
    decoded_sentence = ""
    
    while True :
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        
        decoded_sentence += sampled_char
        
        if sampled_char == '\n' or len(decoded_sentence) > max_tar_len : #\n==<eos>
            break
        
        #saving predicted value
        target_seq = np.zeros((1,1,tar_vocab_size))
        target_seq[0,0, sampled_token_index] = 1
        
        #saving states
        states_value = [h, c]
    
    return decoded_sentence

# Test

In [35]:
input_seq = encoder_input[3:4]
print(input_seq.shape)
print(input_seq)
states_value = encoder_model.predict(input_seq)
print(states_value)

(1, 31, 81)
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]
1/1 [==============================] - 1s 770ms/step
[array([[ 4.18275548e-03,  6.25762017e-03, -4.00721729e-02,
         9.36522558e-02, -2.34091394e-02, -2.78388739e-01,
         3.88122140e-03, -8.01314175e-01, -1.00321598e-01,
         4.64301631e-02, -3.44036031e-03,  2.60620634e-03,
        -1.99010456e-03, -6.36759937e-01, -3.46003324e-02,
        -3.77164446e-02,  9.27644491e-01, -2.56222603e-03,
         1.25499025e-01, -5.01221232e-02,  1.39225990e-01,
        -1.29396558e-01,  2.02167630e-02, -9.28613415e-04,
         5.24788164e-02, -4.51589197e-01, -7.49791414e-03,
        -6.60956046e-03,  5.90994656e-01,  8.10001940e-02,
        -9.09691211e-03, -1.13071315e-01, -7.17031538e-01,
        -8.32844198e-01,  5.43105416e-03, -5.71421981e-01,
        -4.58579212e-02, -5.08025348e-01, -1.50266793e-02,
   

In [36]:
target_seq = np.zeros((1,1,tar_vocab_size))
target_seq[0,0, tar_to_index['\t']] = 1
print(np.array(target_seq).shape)
print(np.array(states_value).shape)
print([target_seq] + states_value)

(1, 1, 100)
(2, 1, 256)
[array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]]]), array([[ 4.18275548e-03,  6.25762017e-03, -4.00721729e-02,
         9.36522558e-02, -2.34091394e-02, -2.78388739e-01,
         3.88122140e-03, -8.01314175e-01, -1.00321598e-01,
         4.64301631e-02, -3.44036031e-03,  2.60620634e-03,
        -1.99010456e-03, -6.36759937e-01, -3.46003324e-02,
        -3.77164446e-02,  9.27644491e-01, -2.56222603e-03,
         1.25499025e-01, -5.01221232e-02,  1.39225990e-01,
        -1.29396558e-01,  2.02167630e-02, -9.28613415e-04,
         5.24788164e-02, -4.51589197

In [37]:
output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

sampled_token_index = np.argmax(output_tokens[0, -1, :])
print(sampled_token_index)

1/1 [==============================] - 0s 223ms/step
49


In [92]:
for seq_index in [3,50,100,300,1001] : 
    input_seq = encoder_input[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('Input :', lines.src[seq_index])
    print('Correct answer:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1])
    print('Output :', decoded_sentence[0:len(decoded_sentence)-1]) 

1/1 [==============================] - 0s 18ms/step
-----------------------------------
Input : Go.
Correct answer: Váyase.
Output : Vete a Mary.
1/1 [==============================] - 0s 17ms/step
-----------------------------------
Input : Shoot!
Correct answer: ¡Disparad!
Output : ¡Dispare!
1/1 [==============================] - 0s 28ms/step
-----------------------------------
Input : I wept.
Correct answer: Lloré.
Output : Me respeta.
1/1 [==============================] - 0s 31ms/step
-----------------------------------
Input : Who ran?
Correct answer: ¿Quién corría?
Output : ¿Quién corrió?
1/1 [==============================] - 0s 25ms/step
-----------------------------------
Input : Stay home.
Correct answer: Quédate en casa.
Output : Quédate aquí.


In [90]:
def text_input_preprocessing(input_sentence) :
    incoded = []
    
    for word in input_sentence :
        incoded.append(src_to_index[word])
        
    incoded = pad_sequences([incoded], maxlen=max_src_len, padding="post")
    print(incoded)
    test_input = to_categorical(incoded, num_classes=81)
    
    return test_input

In [98]:
test = "I am a student."

test_input = text_input_preprocessing(test)
print(test_input.shape)
print(test_input)
print(decode_sequence(test_input))

[[31  1 49 61  1 49  1 67 68 69 52 53 62 68  9  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]]
(1, 31, 81)
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]
1/1 [==============================] - 0s 31ms/step
Soy un estudiante.

